# Майнор по Анализу Данных, Группа ИАД-4
## 23/11/2017 Оценка доверительных интервалов, бутстреп

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')

%matplotlib inline

RND_SEED = 1234

### Доверительный интервал для доли

Например, был проведен опрос, на котором респондентам задавали вопрос с бинарным ответом, который был закодирован `0` и `1` (Представим, что все люди честные). Требуется оценить доверительный интервал доли ответов.

In [ ]:
np.random.seed(RND_SEED)

In [ ]:
# Сначала генеральная совокупность
statistical_population = np.random.randint(2, size = 100000) 

# Теперь наша выборка
random_sample = np.random.choice(statistical_population, size = 1000)

In [ ]:
# Истинное значение доли
statistical_population.mean()

In [ ]:
# Точечная оценка доли 
random_sample.mean()

In [ ]:
# Хотим доверительный интервал!
from statsmodels.stats.proportion import proportion_confint

#### Доверительный интервал на основе нормального распределения

$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [ ]:
normal_interval = proportion_confint(random_sample.sum(), random_sample.shape[0], method = 'normal', alpha=0.05)
normal_interval

In [ ]:
# Тоже самое только своими руками
def mean_confidence_interval(data, alpha=0.05):
    # Your code here

In [ ]:
mean_confidence_interval(random_sample)

#### Доверительный интервал Уилсона

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{
z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [ ]:
wilson_interval = proportion_confint(random_sample.sum(), random_sample.shape[0], method = 'wilson')
wilson_interval

#### Размер выборки для интервала заданной ширины

In [ ]:
from statsmodels.stats.proportion import samplesize_confint_proportion

In [ ]:
n_samples = int(np.ceil(samplesize_confint_proportion(random_sample.mean(), 0.01)))
n_samples

In [ ]:
np.random.seed(1)
random_sample = np.random.choice(statistical_population, size = n_samples)

In [ ]:
normal_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'normal')
normal_interval[1] - normal_interval[0]

### Доверительный интервал для двух долей

Пускай у нас есть рекламные баннеры. Мы хотим сравнить их "кликабельность"

In [ ]:
data = pd.read_csv('banner_click_stat.txt', header = None, sep = '\t')
data.columns = ['banner_a', 'banner_b']

In [ ]:
data.head()

In [ ]:
data.describe()

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [ ]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    # Your code here
    return (left_boundary, right_boundary)   

In [ ]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(data.banner_a, data.banner_b)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [ ]:
pd.crosstab(data.banner_a, data.banner_b)

In [ ]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = sp.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [ ]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_rel(data.banner_a, data.banner_b)

## Доверительные интервалы для средних

Рассмотрим некий набор данных.

* Выберем два алгоритма классификации.
* Запустим кросс-валидацию с оценкой AUC
* Сравним средние AUC и их доверительные интервалы

In [ ]:
from sklearn import cross_validation, datasets, linear_model, metrics
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X, y = datasets.make_blobs(300, centers=2, cluster_std=3, random_state=RND_SEED)

In [ ]:
plt.scatter(X[:,0], X[:,1], c=y, s=50)

Выберите два алгоритма и оцените их AUC на каждой итерации кросс-валидации

In [ ]:
model1 = linear_model.LogisticRegression()
model2 = RandomForestClassifier()

In [ ]:
scores1 = cross_validation.cross_val_score(model1, X, y, scoring='roc_auc', cv=20)
scores2 = cross_validation.cross_val_score(model2, X, y, scoring='roc_auc', cv=20)

In [ ]:
print scores1.mean()
print scores2.mean()

Доверительные интервалы для среднего можно оценить двумя способами.

Допустим, нам откуда-то известно, что дисперсия auc_scores $\sigma^2=0.25$. 
Построим доверительные интервалы для средних вида $$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

Вместо гипотетической теоретической дисперсии $\sigma^2$, которую мы на самом деле в данном случае не знаем, можно использовать выборочные дисперсии, и построить доверительные интервалы вида $$\bar{X}_n \pm t_{1-\frac{\alpha}{2}} \frac{S}{\sqrt{n}}$$

In [ ]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [ ]:
n = 20
print _zconfint_generic(scores1.mean(), np.sqrt(0.25/n), alpha=0.05, alternative='two-sided')
print _zconfint_generic(scores2.mean(), np.sqrt(0.25/n), alpha=0.05, alternative='two-sided')

In [ ]:
se1 = scores1.std(ddof=1)/np.sqrt(n)
se2 = scores2.std(ddof=1)/np.sqrt(n)

In [ ]:
print _tconfint_generic(scores1.mean(), se1, alpha=0.05, dof=n-1, alternative='two-sided')
print _tconfint_generic(scores1.mean(), se2, alpha=0.05, dof=n-1, alternative='two-sided')

## Доверительный интервал статистик на основе bootstrap

Verizon — основная региональная телекоммуникационная компания (Incumbent Local Exchange Carrier, ILEC) в западной 
части США. В связи с этим данная компания обязана предоставлять сервис ремонта телекоммуникационного оборудования 
не только для своих клиентов, но и для клиентов других локальных телекоммуникационых компаний (Competing Local Exchange Carriers, CLEC). При этом в случаях, когда время ремонта оборудования для клиентов других компаний существенно выше, чем для собственных, Verizon может быть оштрафована. 

In [ ]:
data = pd.read_csv('verizon.txt', sep='\t')
data.shape

In [ ]:
data.Group.value_counts()

С помощью бутстрепа оцените 95% доверительные интервалы для медианы в каждой из категорий, а так же для разности медиан

In [ ]:
sample1 = data.query('Group == "ILEC"').loc[:,'Time'].values
sample2 = data.query('Group != "ILEC"').loc[:,'Time'].values

In [ ]:
sample1.shape

In [ ]:
print sample1.mean()
print sample2.mean()

In [ ]:
plt.hist(sample1, bins=20)

In [ ]:
plt.hist(sample2, bins=20)

In [ ]:
def bootstrap(sample, m=1000, func=np.median, level=0.05):
    # Your code here

## Доверительный интервал коэффициентов регрессии

С помощью бутстрепа оцените доверительный интервал коэффициентов регрессии на данных

In [ ]:
from sklearn import datasets

In [ ]:
X, y = datasets.make_regression(n_samples=100, n_features=1, n_informative=1,
                                n_targets=1, bias=2.0, tail_strength=0.5, noise=150, shuffle=True, random_state=RND_SEED)

In [ ]:
plt.scatter(X[:,0], y)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Your Code Here

In [ ]:
import seaborn as sns